In [1]:
import { ChatDeepSeek } from '@langchain/deepseek';
import { HumanMessage } from "@langchain/core/messages";

const model = new ChatDeepSeek(
    {
        model:'deepseek-chat'
    }
);

await model.invoke([
    new HumanMessage("讲个笑话")
])


AIMessage {
  "id": "d46edae1-a058-4901-81d2-8fa933639624",
  "content": "有一天，小明去参加一个面试。面试官问他：“你有什么特长吗？”\n\n小明想了想，认真地回答：“我会预测未来。”\n\n面试官很感兴趣，问：“哦？那你能预测一下你什么时候能被录用吗？”\n\n小明微微一笑，说：“这个嘛……我预测我不会被录用。”\n\n面试官愣了一下，笑着说：“你的预测很准，你确实不会被录用。”\n\n小明耸耸肩：“看，我说对了吧！”",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 6,
      "completionTokens": 85,
      "totalTokens": 91
    },
    "finish_reason": "stop",
    "model_name": "deepseek-chat",
    "usage": {
      "prompt_tokens": 6,
      "completion_tokens": 85,
      "total_tokens": 91,
      "prompt_tokens_details": {
        "cached_tokens": 0
      },
      "prompt_cache_hit_tokens": 0,
      "prompt_cache_miss_tokens": 6
    },
    "system_fingerprint": "fp_3a5770e1b4_prod0225"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 85,
    "input_tokens": 6,
    "total_tokens": 91,
    "input_token_details": {
      "cache_read": 0
    },
    "output_token_detail

In [2]:
// StringOutputParser 提取api返回的文本数据
import { StringOutputParser } from "@langchain/core/output_parsers";

const parser = new StringOutputParser();


const chain = model.pipe(parser)

await chain.invoke([
    new HumanMessage("讲个笑话")
])



"当然可以！这是一个经典的笑话：\n" +
  "\n" +
  "有一天，小明去参加一个面试。面试官问他：“你有什么特长吗？”\n" +
  "\n" +
  "小明想了想，认真地说：“我会模仿动物。”\n" +
  "\n" +
  "面试官有点好奇，说：“那你模仿一下狗吧。”\n" +
  "\n" +
  "小明立刻蹲下来，开始“汪汪汪”地叫。面试官点点头，说：“不错，那你会模仿猫吗？”\n" +
  "\n" +
  "小明又立刻趴在地上，开始“喵喵喵”地叫。\n" +
  "\n" +
  "面试官笑了笑，接着问：“那你还会模仿什么动物？”\n" +
  "\n" +
  "小明想了想，突然站起来，拍了拍桌子，大声说：“老板，我要加薪！”\n" +
  "\n" +
  "面试官：…… 😄"

In [3]:
// 部分 Parser 会内置一些预先设计好的 prompt 对模型进行引导
import { StructuredOutputParser } from "langchain/output_parsers";
import { PromptTemplate } from "@langchain/core/prompts";

const parser = StructuredOutputParser.fromNamesAndDescriptions({
  answer: "用户问题的答案",
  evidence: "你回答用户问题所依据的答案",
  confidence: "问题答案的可信度评分，格式是百分数",
});



In [4]:

console.log(parser.getFormatInstructions())
// 先告诉 LLM 输出的类型
// 然后，使用 few-shot （一种 prompt 技巧），也就是用示例告诉 LLM 什么是 JSON Schema，什么情况会被解析成功，什么情况不会被解析成功
// 然后，再次强调类型的重要性，输出必须遵循给定的JSON Schema实例，确保所有字段严格匹配Schema中的定义，没有额外的属性，也没有遗漏的必需属性。
// 并强调需要注意细节，比如不要在JSON对象中添加多余的逗号，这可能会导致解析失败。
// 这些 prompt 质量非常高，把在该任务中大模型容易出现的问题都进行了强调，可以有效的保证输出的质量。
// 最后才是给出，我们指定的 JSON 格式和对应的描述


You must format your output as a JSON value that adheres to a given "JSON Schema" instance.

"JSON Schema" is a declarative language that allows you to annotate and validate JSON documents.

For example, the example "JSON Schema" instance {{"properties": {{"foo": {{"description": "a list of test words", "type": "array", "items": {{"type": "string"}}}}}}, "required": ["foo"]}}}}
would match an object with one required property, "foo". The "type" property specifies "foo" must be an "array", and the "description" property semantically describes it as "a list of test words". The items within "foo" must be strings.
Thus, the object {{"foo": ["bar", "baz"]}} is a well-formatted instance of this example "JSON Schema". The object {{"properties": {{"foo": ["bar", "baz"]}}}} is not well-formatted.

Your output will be parsed and type-checked according to the provided schema instance, so make sure all fields in your output match the schema exactly and there are no trailing commas!

Here is the JS

In [5]:
const prompt = PromptTemplate.fromTemplate("尽可能的回答用的问题 \n{instructions} \n{question}")
// const model = new ChatDeekSeek();

const chain = prompt.pipe(model).pipe(parser)
const res = await chain.invoke({
    question: "蒙娜丽莎的作者是谁？是什么时候绘制的",
    instructions: parser.getFormatInstructions()
})
                               
console.log(res)


{
  answer: "蒙娜丽莎的作者是列奥纳多·达·芬奇，绘制时间大约在1503年至1506年之间。",
  evidence: "蒙娜丽莎是世界上最著名的肖像画之一，其作者列奥纳多·达·芬奇是文艺复兴时期的杰出艺术家。根据历史记录和艺术史研究，达·芬奇在1503年至1506年间创作了这幅画。",
  confidence: "95%"
}


In [6]:

// List Output Parser,限制列表格式得到指令
import { CommaSeparatedListOutputParser } from "@langchain/core/output_parsers";

const parser = new CommaSeparatedListOutputParser();

console.log(parser.getFormatInstructions())


Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [7]:

// const model = new ChatOpenAI();
const prompt = PromptTemplate.fromTemplate("列出3个 {country} 的著名的互联网公司.\n{instructions}")
    
const chain = prompt.pipe(model).pipe(parser)

const response = await chain.invoke({
    country: "America",
    instructions: parser.getFormatInstructions(),
});


In [8]:
response

[ "Google", "Amazon", "Facebook" ]

In [9]:

// Auto Fix Parser ，对于部分对输出质量要求更高的场景，如果出现了输出不符合要求的情况，我们希望的不是让 LLM 反复输出（可能每次都是错的），因为 LLM 并没有意识到自己的错误。所以我们需要把报错的信息返回给 LLM，让他理解错在哪里，应该怎么修改。

import { z } from "zod";
const schema = z.object({
  answer:  z.string().describe("用户问题的答案"),
  confidence: z.number().min(0).max(100).describe("问题答案的可信度评分，满分 100")
});
const parser = StructuredOutputParser.fromZodSchema(schema);
const prompt = PromptTemplate.fromTemplate("尽可能的回答用的问题 \n{instructions} \n{question}")
// const model = new ChatOpenAI();

const chain = prompt.pipe(model).pipe(parser)
const res = await chain.invoke({
    question: "蒙娜丽莎的作者是谁？是什么时候绘制的",
    instructions: parser.getFormatInstructions()
})
                               
console.log(res)


{ answer: "蒙娜丽莎的作者是列奥纳多·达·芬奇，大约在1503年至1506年间绘制。", confidence: 95 }


In [ ]:
// fixParse只是一个纯粹的json更合适处理，可以修复json格式的问题。
import { OutputFixingParser } from "langchain/output_parsers"
// 我们构造一个错误的输出，
const wrongOutput = {
  "answer": "蒙娜丽莎的作者是达芬奇，大约在16世纪初期（1503年至1506年之间）开始绘制。",
  "sources": "90%" 
};
console.log({OutputFixingParser});


const fixParser = OutputFixingParser.fromLLM(model, parser);
const output = await fixParser.parse(JSON.stringify(wrongOutput));
output

{
  OutputFixingParser: [class OutputFixingParser extends BaseOutputParser]
}


{ answer: "蒙娜丽莎的作者是达芬奇，大约在16世纪初期（1503年至1506年之间）开始绘制。", confidence: 90 }

: 

 出于节约成本考虑，可以实现gpt4的错误输出，用3.5来修复，可以节约成本